In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from deepface import DeepFace
from mtcnn import MTCNN
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import random
import joblib


def enhance_face(input_image_path, scale_factor=6, denoise_strength=6):
    try:
        img = cv2.imread(input_image_path)
        denoised_img = cv2.fastNlMeansDenoisingColored(
            img, None, denoise_strength, denoise_strength, 7, 21
        )
        upscaled_img = cv2.resize(
            denoised_img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR
        )
        return upscaled_img
    except Exception as e:
        print(f"Error enhancing image: {e}")
        return None


def generate_vgg_face_embeddings(images):
    embeddings = []
    for img in images:
        try:
            result = DeepFace.represent(img, model_name="VGG-Face", enforce_detection=False)
            if result and "embedding" in result[0]:
                embeddings.append(result[0]["embedding"])
            else:
                print("No embedding found for this image.")
        except Exception as e:
            print(f"Error processing image: {e}")
    return embeddings


def load_images_and_labels(folder):
    images = []
    labels = []
    names = os.listdir(folder)
    for label in names:
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            img = enhance_face(img_path)
            if img is not None:
                images.append(img)
                labels.append(label)
    le = LabelEncoder()
    encoded_labels = le.fit_transform(labels)
    return images, encoded_labels, le, names


def save_model(clf, label_encoder, model_path, encoder_path):
    joblib.dump(clf, model_path)
    joblib.dump(label_encoder, encoder_path)


def load_model(model_path, encoder_path):
    clf = joblib.load(model_path)
    label_encoder = joblib.load(encoder_path)
    return clf, label_encoder


def train_and_save_model(folder, model_path="model.pkl", encoder_path="label_encoder.pkl"):
    train_images, train_labels, label_encoder, student_names = load_images_and_labels(folder)
    train_embeddings = generate_vgg_face_embeddings(train_images)

    combined = list(zip(train_embeddings, train_labels))
    random.shuffle(combined)
    train_embeddings, train_labels = zip(*combined)
    train_embeddings = np.array(train_embeddings)
    train_labels = np.array(train_labels)

    clf = SVC(probability=True)
    clf.fit(train_embeddings, train_labels)

    save_model(clf, label_encoder, model_path, encoder_path)
    return student_names


def evaluate_on_test_image(test_image_path, clf, label_encoder, student_names, confidence_threshold=0.3):
    enhanced_image = enhance_face(test_image_path)
    faces = detect_faces_with_mtcnn(enhanced_image)

    recognized_names = set()
    for face in faces:
        embedding = DeepFace.represent(face, model_name="VGG-Face", enforce_detection=False)[0]["embedding"]
        prediction = clf.predict_proba([embedding])[0]
        predicted_label_idx = np.argmax(prediction)
        confidence = prediction[predicted_label_idx]
        if confidence >= confidence_threshold:
            predicted_label = label_encoder.inverse_transform([predicted_label_idx])[0]
        else:
            predicted_label = "Other"
        recognized_names.add(predicted_label)

    attendance = {name: "Absent" for name in student_names}
    for name in recognized_names:
        attendance[name] = "Present"

    return attendance


def detect_faces_with_mtcnn(image):
    detector = MTCNN()
    results = detector.detect_faces(image)
    faces = []
    for result in results:
        x, y, w, h = result['box']
        face = image[y:y+h, x:x+w]
        faces.append(face)
    return faces


def write_attendance_to_excel(attendance, output_path):
    df = pd.DataFrame(list(attendance.items()), columns=["Name", "Attendance"])
    df.to_excel(output_path, index=False)




24-05-05 00:56:33 - Directory C:\Users\Lailoma Noor/.deepface created
24-05-05 00:56:33 - Directory C:\Users\Lailoma Noor/.deepface/weights created


In [3]:

folder = "face_data"
test_image_path = "test_image.jpeg"
model_path = "model.pkl"
encoder_path = "label_encoder.pkl"
output_path = "attendance.xlsx"

student_names = train_and_save_model(folder, model_path, encoder_path)
clf, label_encoder = load_model(model_path, encoder_path)



24-05-05 01:01:35 - vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: C:\Users\Lailoma Noor\.deepface\weights\vgg_face_weights.h5
 11%|█         | 64.5M/580M [43:13<3:29:51, 40.9kB/s] 

KeyboardInterrupt: 

In [27]:
test_image_path = "/content/other.jpeg"
model_path = "/content/model.pkl"
encoder_path = "/content/label_encoder.pkl"
output_path = "/content/attendance1.xlsx"

clf, label_encoder = load_model(model_path, encoder_path)
student_names = [label for label in label_encoder.classes_]
attendance = evaluate_on_test_image(test_image_path, clf, label_encoder, student_names)
write_attendance_to_excel(attendance, output_path)

print(f"Attendance saved to {output_path}")


1/1 [==============================] - 1s 608ms/step
Attendance saved to /content/attendance1.xlsx
